In [27]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
import tensorflow_hub as hub
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
import numpy as np
import pandas as pd
import numpy as np
import nltk
import re

In [28]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
data = pd.read_csv("C:\__DESKTOP\GitHub\Political-Sentiment-Analysis\word2vec\Twitter_Data.csv")

data = shuffle(data)
data.reset_index(inplace=True, drop=True)
data = data[data.category!= 0]
data = data.sort_values(by=['clean_text'])
data = data[:-1190]
data = shuffle(data)
data = data.head(20000)

In [56]:
X = []
for sen in data['clean_text']:
  X.append(sen)
Y = np.array((data['category']+1)/2,dtype=np.int64)

In [57]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [33]:
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [47]:
def preprocess(text):
    text = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
          tokens.append(token)
    return " ".join(tokens)

def encoding(tweet):
  tweet = preprocess(tweet)
  tokens = tweet.split(" ")
  embedding = []
  for token in tokens:
    embedding.append(embed([token]).numpy()[0])

  embedding = np.array(embedding)
  padded_tweet = padding(embedding)
  return padded_tweet

def padding(encoded_tweet):
  count = 70 - encoded_tweet.shape[0]
  pad = np.zeros((1,250))
  for i in range(count):
    encoded_tweet = np.concatenate((pad,encoded_tweet), axis=0)
  return encoded_tweet


def processed_tweets(tweets):
  x = []
  for tweet in tweets:
    enc = encoding(tweet)
    x.append(enc)
  x = np.array(x)
  return x

In [58]:
X_train = processed_tweets(X_train)
X_test = processed_tweets(X_test)

In [49]:
print(type(X_train))
print(X_test.shape)
print(X_train.shape)

<class 'numpy.ndarray'>
(200, 70, 250)
(800, 70, 250)


In [50]:
tweet = ['Pained by the loss of lives due to a fire at a hospital in Bharuch. Condolences to the bereaved families.','To add further momentum to our strong strategic partnership, President Putin and I have agreed to establish a Ministerial Dialogue between our Foreign and Defence Ministers.']
x = processed_tweets(tweet)
y = np.array([0,1])
print(type(x[0][0][0]))
print(x.shape)

<class 'numpy.float64'>
(2, 70, 250)


In [54]:
clf = Sequential()

clf.add(LSTM(128, activation='relu', return_sequences=True))
clf.add(Dropout(0.2))
clf.add(LSTM(64, activation='relu'))
clf.add(Dropout(0.2))
clf.add(Dense(32, activation='relu'))
clf.add(Dropout(0.2))
clf.add(Dense(1,activation='sigmoid'))

optimizer = optimizers.Adam(clipvalue=0.5)
clf.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [66]:
clf.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=2)

Epoch 1/2
500/500 [==============================] - 50s 100ms/step - loss: 0.1510 - accuracy: 0.9401 - val_loss: 6880738397913088.0000 - val_accuracy: 0.8240
Epoch 2/2
500/500 [==============================] - 43s 86ms/step - loss: 0.1858 - accuracy: 0.9296 - val_loss: 5823969747795968.0000 - val_accuracy: 0.8242


In [67]:
clf.predict(processed_tweets(tweet))

array([[0.17189175],
       [0.9977386 ]], dtype=float32)

In [62]:
clf.save('model83.h5')